In [1]:
import pandas as pd

from core.db.Terraz import TerrazDB
from terraz_ingestor import TerrazIngestor

# Análise de qualidade do Banco de Dados

Esse relatório tem como objetivo analisar os dados presentes no banco de dados da Terraz disponibilizado e também o arquivo com os leads e informações dos clientes e negócios.

## Inicializando DB

In [11]:
terraz = TerrazDB()

## Verificando porcentagem de nulos por tabela disponibilizada

### public.terraz_contacts

In [12]:
query_str = """
select key as column, round(avg(case when value is null then 1 else 0 end)*100, 2) as null_values
from terraz_contacts t
cross join jsonb_each_text(to_jsonb(t))
group by key;
"""

terraz.query_to_pandas(query_str)

,column,null_values
0,ADDRESS,99.92
1,ADDRESS_2,100.00
2,ADDRESS_CITY,100.00
3,ADDRESS_COUNTRY,100.00
4,ADDRESS_COUNTRY_CODE,100.00
...,...,...
166,UTM_CONTENT,99.90
167,UTM_MEDIUM,99.82
168,UTM_SOURCE,99.82
169,UTM_TERM,99.90


### public.terraz_leads

In [ ]:
query_str = """
select key as column, round(avg(case when value is null then 1 else 0 end)*100, 2) as null_values
from terraz_leads t
cross join jsonb_each_text(to_jsonb(t))
group by key;
"""

terraz.query_to_pandas(query_str)

### terraz_bitrix.deals

In [14]:
query_str = """
select key as column, round(avg(case when value is null then 1 else 0 end)*100, 2) as null_values
from terraz_bitrix.deals t
cross join jsonb_each_text(to_jsonb(t))
group by key;
"""

terraz.query_to_pandas(query_str)

,column,null_values
0,ADDITIONAL_INFO,99.87
1,ASSIGNED_BY_ID,0.00
2,BEGINDATE,0.00
3,CATEGORY_ID,0.00
4,CLOSED,0.00
...,...,...
885,UTM_CAMPAIGN,98.93
886,UTM_CONTENT,99.08
887,UTM_MEDIUM,98.92
888,UTM_SOURCE,98.92


### terraz_bitrix.contacts

In [15]:
query_str = """
select key as column, round(avg(case when value is null then 1 else 0 end)*100, 2) as null_values
from terraz_bitrix.contacts t
cross join jsonb_each_text(to_jsonb(t))
group by key;
"""

terraz.query_to_pandas(query_str)

,column,null_values
0,ADDRESS,99.86
1,ADDRESS_2,99.94
2,ADDRESS_CITY,99.94
3,ADDRESS_COUNTRY,99.94
4,ADDRESS_COUNTRY_CODE,100.00
...,...,...
162,UTM_CONTENT,99.85
163,UTM_MEDIUM,99.76
164,UTM_SOURCE,99.76
165,UTM_TERM,99.84


Como é possível observar, a quantidade de valores nulos são bem elevadas e os nomes de variáveis são pouco auto explicativos, dificultando o uso desse banco de dados em um primeiro momento.

## Arquivos enviados por Google Drive

In [2]:
terraz_ing = TerrazIngestor()
leads_df = terraz_ing.main(False, False)
leads_df.isna().mean() * 100

/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_bytes, sep=r'\t', encoding='latin-1', on_bad_lines='skip')
/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_bytes, sep=r'\t', encoding='latin-1', on_bad_lines='skip')
/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:29: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 c

ID                                                         0.000000
Etapa                                                      0.000000
Nome do Lead                                               0.004994
Saudação                                                  99.999773
Primeiro nome                                              8.668866
                                                            ...    
Empresa CRM                                               63.308961
Última atualização em                                      6.622870
Criação de deal                                           99.991601
Modelos                                                  100.000000
Terraz - Preencher campo 'imobiliária' / Qualificação     56.847945
Length: 139, dtype: float64

## Gerando amostra

Primeiro removemos as variáveis com menos de 10% de dados disponíveis.

In [3]:
leads_df1 = terraz_ing.missing_cleaner(leads_df)
leads_df1 = leads_df1[leads_df1['Total'] > 0]
# leads_df1 = leads_df1[leads_df1['Lead repetido'] == 'N']
# leads_df1 = leads_df1[~leads_df1['Canal [!]'].isna()]
leads_df.shape, leads_df1.shape

((440519, 139), (171254, 53))

In [13]:
leads_df1.isna().mean()

ID                                                       0.000000
Etapa                                                    0.000000
Nome do Lead                                             0.000029
Primeiro nome                                            0.025126
Criado                                                   0.000000
Fonte                                                    0.000654
Telefone de trabalho                                     0.087753
Email de trabalho                                        0.001857
Responsável                                              0.000000
Criado por                                               0.000000
Modificado                                               0.000000
Modificado por                                           0.000000
Comentário                                               0.854608
Total                                                    0.000000
Moeda                                                    0.000000
Lead repet

In [4]:
leads_df1['visited'] = leads_df1['Data e hora da visita [!]'].apply(lambda x: 1 if pd.isna(x) else 0)
leads_df1['desistiu'] = leads_df1['Data e Hora - Desistiu [!]'].apply(lambda x: 1 if pd.isna(x) else 0)

Após isso, transformamos as variáveis categóricas em variáveis indicadoras com cada categoria disponível. Além disso, as variáveis de data são separadas em seus períodos, como dia, mes, ano, hora, minuto, período do dia e estação do ano. Para a localidade está sendo disponibilizado o CEP3.

In [9]:
# leads_df2 = leads_df1[usable_cols]
leads_df2 = terraz_ing.column_names_fixer(leads_df1)
leads_df2 = terraz_ing.eng_vars(leads_df2)

/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Criado'] = pd.to_datetime(df_final['Criado'], format='%d/%m/%Y %H:%M', errors='coerce')
/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['dia'] = df_final['Criado'].dt.day
/home/danielviegas/PycharmProjects/supermind_ingestor/terraz_ingestor.py:50: SettingWithCopyWarning: 
A value is trying to b

In [10]:
leads_df2.shape

(171166, 65)

In [12]:
leads_df2.head()

4       Campanha de marketing
5                   Imovelweb
6                   Imovelweb
7                   Grupo Zap
10    Reconhecimento da marca
Name: Fonte, dtype: object

In [11]:

leads_df2['Fonte'] = leads_df2['Fonte'].fillna('Missing')
leads_df2['Categoria Imovel'] = leads_df2['Categoria Imovel'].fillna('Missing')
leads_df2['Canal'] = leads_df2['Canal'].fillna('Missing')

leads_df2.drop(['periodo_visita', 'CEP'], axis=1, inplace=True)
leads_df2.isna().sum()

/tmp/ipykernel_9709/3170910646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leads_df2['Fonte'] = leads_df2['Fonte'].fillna('Missing')
/tmp/ipykernel_9709/3170910646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leads_df2['Categoria Imovel'] = leads_df2['Categoria Imovel'].fillna('Missing')
/tmp/ipykernel_9709/3170910646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

ID                  0
Etapa               0
Nome do Lead        5
Primeiro nome    4301
Criado              0
                 ... 
dia_da_semana       0
season              0
hora_visita         0
minuto_visita       0
cep3                0
Length: 63, dtype: int64

In [12]:
leads_df2.to_csv('../data/terraz/processed_leads/leads.csv', index=False)